# Phase de scraping

## Objectifs

* Scraper le maximum d'annonces immobilières pour les 11 plus grandes villes du Sénégal (Dakar, Pikine, Touba, Guédiawaye, Kaolack, Mbour, Saint-Louis, Rufisque, Ziguinchor et Diourbel) ;
* Récupérer les données suivantes : la date du scraping, la surface, le nombre de pièces, le prix, le lien internet de l'annonce, l'adresse, la ville, la région, le descriptif de l'annonce, la mention du caractère neuf ou ancien, la date de construction, la nature du bien et les taxes ou frais.

**La liste des sites** : immobilier.sn, senegalcity.com, mamaison.sn, 1000-annonces, senegalaisedelimmobilier.net, immolagune.com, deuxia.com, senegal-locationvente.com et deuxia.com. 

## Senegalcity.com

In [50]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Chrome
from requests import get
import itertools
import pandas as pd
import base64
import time
import re
from datetime import date

In [ ]:
url_senegalcity="http://www.senegalcity.com/annonces.php?pageFrom="

url_achat_1000='https://www.1000-annonces.com/rechercher.php?ty=&tr=0&rg=33&dp=&cc=2&sc=13&kw=&iv=0&pmin=0&pmax=0&lssc=130-131&lp=99-99-0-0&pg='

url_location_1000='https://www.1000-annonces.com/rechercher.php?ty=&tr=0&rg=33&dp=&cc=2&sc=9&kw=&iv=0&pmin=0&pmax=0&lssc=104-105&lp=99-99-0-0-0&pg='

pages_test=2

In [52]:
def get_annonces_senegalcity(start_url,pages):
    driver = webdriver.Chrome()
    data={}
    df=pd.DataFrame(columns=['Links','Title','Type_de_bien','Nb_pieces','Surface_m2','Prix','Description'])
    for current_page in list(range(pages)):
        driver.get(start_url+str(current_page))# pour tourner les pages
        annonces=driver.find_elements_by_class_name("cadre-blanc")# contenu de chaque annonce
        del(annonces[0])# supprime la premiere annonce de chaque page qui sert de titre à chaque page
        for annonce in annonces:
            try:
                data['Links']=annonce.find_element_by_css_selector('a').get_attribute('href')
                data['Title']=annonce.find_element_by_css_selector('td:nth-child(2) > a').text
                characteristics=annonce.find_elements_by_class_name('annonce-list-item')
                data['Type_de_bien']=characteristics[0].text
                data['Nb_pieces']=characteristics[1].text
                data['Surface_m2']=characteristics[2].text
                data['Prix']=characteristics[3].text
                data['Description']=annonce.find_element_by_css_selector('table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td').text
            except:
                data['Description']=None
            df=df.append(data,ignore_index=True)
    driver.quit()        
    return df                            

In [54]:
data_senegalcity=get_annonces_senegalcity(url_senegalcity,pages_test)

In [55]:
data_senegalcity

,Links,Title,Type_de_bien,Nb_pieces,Surface_m2,Prix,Description
0,http://www.senegalcity.com/annonce.php?idImmob...,Location de vacances Appartement - Point E (Da...,Appartement,4 pièces,300m²,1 800 000 F CFA (2 744 €) / mois,Un très bel appartement meuble à louer avec un...
1,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Ouakam (Dakar) - 4 pièc...,Appartement,4 pièces,150m²,300 000 F CFA (457 €) / mois,Un appartement à louer à Ouakam il compose de ...
2,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Point E (Dakar) - 4 piè...,Appartement,4 pièces,300m²,1 500 000 F CFA (2 287 €) / mois,Un très bel appartement à louer avec ascenseur...
3,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Sipress (Dakar) - 1 piè...,Appartement,1 pièce,60m²,200 000 F CFA (305 €) /,"Studio meublé composé d’une pièce climatisée ,..."
4,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Ouakam (Dakar) - 3 pièc...,Appartement,3 pièces,120m²,260 000 F CFA (396 €) /,Appartement spacieux au Rdc composé de 2ch ( c...
...,...,...,...,...,...,...,...
205,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Ouest Foire (Dakar) - 4...,Appartement,4 pièces,60m²,25 000 F CFA (38 €) /,Bel appartement (rez-de-chaussée) Lumineux sit...
206,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Ouakam (Dakar) - 4 pièc...,Appartement,4 pièces,150m²,400 000 F CFA (610 €) /,appartement de 4 chambres salon cuisine toilet...
207,http://www.senegalcity.com/annonce.php?idImmob...,Location Appartement - Sacré Cœur (Dakar) - 4 ...,Appartement,4 pièces,120m²,550 000 F CFA (838 €) /,tres bel appartement de 3 chambres salon cuisi...
208,http://www.senegalcity.com/annonce.php?idImmob...,Vente Villa - Warang (Thiès) - 8 pièces 1000 m²,Villa,8 pièces,1000m²,99 750 000 F CFA (152 068 €),Auberge à vendre à Warang 5 chambres 32 m² et\...


In [56]:
# Extraction
#data_senegalcity.to_excel (r'C:\Users\jeanc\Documents\fac\M2\python_1\Projet_immobilier_BF\export_senegalcity_scraping.xlsx',sheet_name='01012021', index = False, header=True)

## 1000-annonces.com

In [65]:
def get_links_1000(start_url,pages):
    driver = webdriver.Chrome()
    liste_liens=[]
    for current_page in list(range(pages)):
        driver.get(start_url+str(current_page))# pour tourner les pages
        panel_annonce=driver.find_element_by_class_name("column.is-three-quarters")
        annonces = panel_annonce.find_elements_by_class_name("card-image")
        for annonce in annonces:
            liste_liens.append(annonce.find_element_by_css_selector('a').get_attribute('href'))
    driver.quit()
    return liste_liens 

def get_annonces_1000(liste_liens):
    driver = webdriver.Chrome()
    data={}
    df=pd.DataFrame(columns=['Links','Title','Prix','Ville','Nb_pieces','Surface_m2','Categorie','Classe_energie','Emission_GES','Frais_compris','Description'])
    for lien in liste_liens:
        driver.get(lien)# pour accéder à l'offre
        panel_annonce=driver.find_element_by_class_name("column.is-three-quarters")# contenu de chaque annonce
        data['Links']=lien
        data['Title']=panel_annonce.find_element_by_class_name('is-uppercase.annonce-title').text
        characteristics_panel=panel_annonce.find_element_by_class_name('columns.is-multiline.is-gapless')
        characteristics=characteristics_panel.find_elements_by_class_name('column.is-8.annonce-property')
        data['Prix']=characteristics[0].text
        data['Ville']=characteristics[1].text
        data['Categorie']=characteristics[2].text
        data['Surface_m2']=characteristics[3].text
        data['Nb_pieces']=characteristics[4].text
        data['Classe_energie']=characteristics[5].text
        data['Emission_GES']=characteristics[6].text
        try:
            data['Frais_compris']=characteristics[7].text
        except:
            data['Frais_compris']=None
        data['Description']=panel_annonce.find_element_by_class_name('column.is-12.annonce-description').text
        df=df.append(data,ignore_index=True)
    driver.quit
    return df 


In [66]:
Liens_test1=get_links_1000(url_achat_1000,pages_test)
Scraping_1000=get_annonces_1000(Liens_test1)

In [67]:
Scraping_1000

,Links,Title,Prix,Ville,Nb_pieces,Surface_m2,Categorie,Classe_energie,Emission_GES,Frais_compris,Description
0,https://www.1000-annonces.com/vente-immobilier...,RESIDENCE CAP SKIRRING,305000€,Cap Skirring | Sénégal,5,1050 m²,Vente immobilier > Maison,non concerné,non concerné,oui,A VENDRE CAP SKIRRING\n\nSUR TERRAIN DE 1050M2...
1,https://www.1000-annonces.com/vente-immobilier...,MAISON À VENDRE DAKAR-PARCELLES ASSAINIES BIEN...,47€,Dakar | Sénégal,4,150 m²,Vente immobilier > Maison,C (de 91 à 150),C (de 11 à 20),oui,A vendre maison de 150m² avec garage aux Parce...
2,https://www.1000-annonces.com/vente-immobilier...,APPARTEMENT NEUF A VENDRE A FANN RÉSIDENCE,400000€,Dakar | Sénégal,7,205 m²,Vente immobilier > Appartement,non concerné,non concerné,None,Appartement Neuf a vendre a Fann Résidence\n\n...
3,https://www.1000-annonces.com/vente-immobilier...,MAISON JOAL-FADIOUTH,15000€,Joal Fadiouth | Sénégal,5,300 m²,Vente immobilier > Maison,non concerné,non concerné,None,Maison situé à Joal-Fadiouth sur la petite côt...
4,https://www.1000-annonces.com/vente-immobilier...,APPARTEMENT NEUF À LOUER ET À VENDRE À FANN-ME...,584616€,Dakar | Sénégal,9,300 m²,Vente immobilier > Appartement,non concerné,non concerné,None,APPARTEMENT NEUF À LOUER ET À VENDRE À FANN-ME...
...,...,...,...,...,...,...,...,...,...,...,...
605,https://www.1000-annonces.com/vente-immobilier...,VILLA À VENDRE,-,Thies | Sénégal,1,nc,Vente immobilier > Maison,nc,nc,non,villa de plain pied à vendre à la cité malick ...
606,https://www.1000-annonces.com/vente-immobilier...,VILLA À VENDRE,-,Thies | Sénégal,1,nc,Vente immobilier > Maison,nc,nc,non,villa R+1 à vendre à thiès à la cité ouvrière ...
607,https://www.1000-annonces.com/vente-immobilier...,VILLA À VENDRE À SENDOU,-,Thies | Sénégal,1,nc,Vente immobilier > Maison,nc,nc,non,villa à vendre à sendou pied dans l'eau avec u...
608,https://www.1000-annonces.com/vente-immobilier...,VILA À VENDRE,-,Thies | Sénégal,1,nc,Vente immobilier > Maison,nc,nc,non,villa à vendre à grand standing quarier calme ...


In [68]:
# Extraction
#Scraping_1000.to_excel (r'C:\Users\jeanc\Documents\fac\M2\python_1\Projet_immobilier_BF\export_1000_scraping.xlsx',
#                       sheet_name='01012021', index = False, header=True)

In [70]:
def get_annonces_location_1000(liste_liens):
    driver = webdriver.Chrome()
    data={}
    df=pd.DataFrame(columns=['Links','Title','Prix','Ville','Nb_pieces','Surface_m2','Categorie','Meuble','Classe_energie','Emission_GES','Description'])
    for lien in liste_liens:
        driver.get(lien)# pour accéder à l'offre
        panel_annonce=driver.find_element_by_class_name("column.is-three-quarters")# contenu de chaque annonce
        data['Links']=lien
        data['Title']=panel_annonce.find_element_by_class_name('is-uppercase.annonce-title').text
        characteristics_panel=panel_annonce.find_element_by_class_name('columns.is-multiline.is-gapless')
        characteristics=characteristics_panel.find_elements_by_class_name('column.is-8.annonce-property')
        data['Prix']=characteristics[0].text
        data['Ville']=characteristics[1].text
        data['Categorie']=characteristics[2].text
        data['Surface_m2']=characteristics[3].text
        data['Nb_pieces']=characteristics[4].text
        data['Meuble']=characteristics[5].text
        data['Classe_energie']=characteristics[6].text
        data['Emission_GES']=characteristics[7].text
        data['Description']=panel_annonce.find_element_by_class_name('column.is-12.annonce-description').text
        df=df.append(data,ignore_index=True)
    driver.quit()
    return df 

In [72]:
Liens_test2=get_links_1000(url_location_1000,pages_test)

In [73]:
Scraping_location_1000=get_annonces_location_1000(Liens_test2)

In [74]:
Scraping_location_1000

,Links,Title,Prix,Ville,Nb_pieces,Surface_m2,Categorie,Meuble,Classe_energie,Emission_GES,Description
0,https://www.1000-annonces.com/location/senegal...,HORS RÉSIDENCE,400€,Saly Portudal | Sénégal,4,110 m²,Location > Maison,oui,non concerné,non concerné,"Saly station ""allo docteur"" à proximité tous c..."
1,https://www.1000-annonces.com/location/senegal...,APPART DE 2 CHAMBRES À LOUER SUR MBOUR,200€,M'Bour | Sénégal,3,120 m²,Location > Appartement,non,non concerné,non concerné,"Disponible sur Mbour en bordure de route, un a..."
2,https://www.1000-annonces.com/location/senegal...,LOUE MAGNIFIQUE APPARTEMENT SITUÉ À MERMOZ,1350€,Dakar | Sénégal,6,100 m²,Location > Appartement,non,non concerné,non concerné,Loue magnifique appartement entièrement neuf s...
3,https://www.1000-annonces.com/location/senegal...,VILLA NEUF F6 À LOUER AU VIRAGE,600€,Dakar | Sénégal,6,400 m²,Location > Maison,non,C (de 91 à 150),D (de 21 à 35),A louer Virage derrière Hotel Onomo\n\nBelle v...
4,https://www.1000-annonces.com/location/senegal...,BELLE VILLA AU LAC ROSE,500000€,Dakar | Sénégal,nc,nc,Location > Maison,non,nc,nc,Une belle villa (R+1) au Lac rose composée de ...
...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.1000-annonces.com/location/senegal...,TRÉS BÉLLE VILLA À LOUER À THIÉS CITÉ MALICK SY,-,Thies | Sénégal,5,600 m²,Location > Maison,non,E (de 231 à 330),D (de 21 à 35),Villa de charme plain pied climatisée dans un ...
186,https://www.1000-annonces.com/location/senegal...,VILLA R+1 À LOUER,-,Thies | Sénégal,1,nc,Location > Maison,non,nc,nc,villa R+1 à louer à mbour 3 à thiès de 3 chbre...
187,https://www.1000-annonces.com/location/senegal...,VILLA À LOUER,-,Thies | Sénégal,1,nc,Location > Maison,non,nc,nc,villa à louer à la Cité Malick sy thiés quarti...
188,https://www.1000-annonces.com/location/senegal...,VILLA À LOUER,-,Thies | Sénégal,1,nc,Location > Maison,non,nc,nc,Mbour 1 trés belle villa neuve R+2 - très bel ...


In [75]:
# Extraction
#Scraping_location_1000.to_excel (r'C:\Users\jeanc\Documents\fac\M2\python_1\Projet_immobilier_BF\export_1000_scraping_loc.xlsx',
#                                 sheet_name='01012021', index = False, header=True)